# 自定义 TOPI 算子

In [1]:
import numpy as np
from tvm import te, topi, tir as T, relay
import tvm
from tvm.topi import tag
from tvm.relay.op.op import register_compute, register_shape_func
from tvm.relay.op.op import register_broadcast_schedule, register_injective_schedule
from tvm.relay.op.op import register_pattern, OpPattern
from tvm.topi.utils import get_const_tuple

@tvm.te.tag_scope(tag=tag.ELEMWISE)
def store_multipliers_shifts(x):
    """存储 multipliers_shifts"""
    # hp_dtype = "int64"
    # lp_dtype = "int32"
    # assert y.dtype == lp_dtype
    # assert left_shift.dtype == lp_dtype
    # assert right_shift.dtype == lp_dtype
    # one = T.const(1, hp_dtype)
    def _compute(*indices):
        # elements = []
        # for element in get_const_tuple(axes):
        #     elements += [indices[element]]
        # param_indices = tuple(elements)

        # 0) 获取值
        value = x(*indices)
        
        return value.astype(x.dtype)

    return te.compute(x.shape, _compute)

In [ ]:
shape = 1, 2
lp_dtype = "int32"
hp_dtype = "int32"
axes = [1]
shift_shape = [shape[ax] for ax in axes]
x = te.placeholder(shape, name="x", dtype=lp_dtype)
y = te.placeholder(shift_shape, name="y", dtype=hp_dtype)
left_shift = te.placeholder(shift_shape, name="left_shift", dtype=hp_dtype)
right_shift = te.placeholder(shift_shape, name="right_shift", dtype=hp_dtype)
# multipliers_shifts = te.placeholder(shape, name="multipliers_shifts", dtype="int32")
z = q_multiply_shift(x, y, 8, left_shift, right_shift, is_left_shift_required=1)
s = te.create_schedule(z.op)
f = tvm.build(s, [x, y, left_shift, right_shift, z], "llvm")
dev = tvm.cpu(0)
a_np = np.ones(shape).astype(x.dtype) * 125
multiplier_np = np.ones(get_const_tuple(y.shape)).astype(hp_dtype) * 36500
ls_np = np.ones(get_const_tuple(left_shift.shape)).astype(hp_dtype) * 8
rs_np = np.ones(get_const_tuple(right_shift.shape)).astype(hp_dtype) * 8
a = tvm.nd.array(a_np, dev) 
multiplier = tvm.nd.array(multiplier_np, dev)
ls = tvm.nd.array(ls_np, dev)
rs = tvm.nd.array(rs_np, dev)
c = tvm.nd.array(np.zeros(get_const_tuple(z.shape), dtype=z.dtype), dev)
f(a, multiplier, ls, rs, c)
print(a, multiplier, ls, rs, c)

In [ ]:
x = relay.var("x", shape=(4, 8), dtype="float32")
op0 = relay.qnn.op.quantize(x, relay.const(2.0), relay.const(10), out_dtype="uint8")
op1 = relay.qnn.op.dequantize(op0, relay.const(0.5), relay.const(5))
relay_mod = tvm.IRModule.from_expr(op1)
relay_mod.show()

In [ ]:
input_name = "data"
shape = (4, 2)
x = relay.var(input_name, shape=shape, dtype="float32")
op0 = relay.qnn.op.quantize(x, relay.const(np.array([2.0, 2.0])), relay.const(np.array([10, 10])), axis=1, out_dtype="uint8")
op1 = relay.qnn.op.dequantize(op0, relay.const(np.array([0.5, 0.5])), relay.const(np.array([5, 5])))
relay_mod = tvm.IRModule.from_expr(op1)
relay_mod.show()

In [ ]:
from tvm_book.tvm_utils.llvm_utils import run_llvm_graph

In [ ]:
from tvm.relay.backend import Executor
data_np = np.ones(shape)
inputs = {input_name: data_np}
cpu_outs = run_llvm_graph(relay_mod, {}, inputs)

In [ ]:
relay_mod.show()